In [0]:
%run "/Workspace/Users/cloud_learning123_outlook.com#ext#@cloudlearning123outlook.onmicrosoft.com/Spark_ETL_End_To_End/Factory_Pattern_Notebooks/Data_Transform_Factory"

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

class Transform():
    """
    Most sold pizza in Chicken Category(Large, Medium, Small) month wise

    """
    def requirementOne(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("Order_Month","name")\
            .orderBy(desc("total_sales"),"size")

        scenarioOneDf =  df\
            .groupBy("name","size","Order_Month","category")\
            .agg(count("order_id").alias("total_sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .filter((col("category")=="Chicken") & (col("Rank") == 1))\
            .selectExpr("name as Pizza_Name","size as Pizza_Size","total_sales","Order_Month")\
            .orderBy("Order_Month",desc("total_sales"))

        print("Most sold pizzas in every chicken pizzas month wise")
        scenarioOneDf.display()
        return scenarioOneDf
    
    """
    Most sold pizza in every Category overall
    """
    def requirementTwo(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("category")\
            .orderBy(desc("total_sales"))

        scenarioTwoDf = df\
            .groupBy("category","size","name")\
            .agg(count("order_id").alias("Total_Sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .filter(col("Rank") == 1)\
            .selectExpr("name as Pizza_Name","size as Pizza_Size","Total_Sales","Category")


        print("Most sold pizza in every Category overall")
        scenarioTwoDf.display()
        return scenarioTwoDf
    
        """
    Most sold pizza during 5.00pm to 10.00pm overall hour wise
    """
    def requirementThree(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec = Window\
            .partitionBy("Order_Hour")\
            .orderBy(desc("Total_Sales"))

        scenarioThreeDf = df\
            .groupBy("Order_Hour","name","size")\
            .agg(count("order_id").alias("Total_Sales"))\
            .withColumn("Rank",rank().over(windowSpec))\
            .filter((col("Rank")==1) & (col("Order_Hour")>16) & (col("Order_Hour")<23))\
            .selectExpr("name as Pizza_Name","size as Size","Total_Sales","Order_Hour")
        print("Most sold pizza during 5.00pm to 10.00pm overall hour wise")
        scenarioThreeDf.display()
        return scenarioThreeDf
    
    def requirementFour(self,inputs):
        df = Derivate.derivate(self,inputs)

        windowSpec =  Window.partitionBy("Order_Month","size").orderBy(desc("Total_Sales"))

        scenarioFourDf = df\
            .groupBy("Order_Month","name","size")\
            .agg(count("order_id").alias("Total_Sales"))\
            .withColumn("Rank", rank().over(windowSpec))\
            .filter((col("Rank") == 1) & (col("size") == "L"))\
            .selectExpr("name as Pizza_Name","Order_Month","Total_Sales")
        
        print("Most Sold Large Pizza month wise ")
        scenarioFourDf.display()
        return scenarioFourDf
    
    def requirementFive(self,inputs):

        """
        Total amount of small Pizza sold in every category
        """
        df = Derivate.derivate(self,inputs)
        
        ScenarioFiveDf = df \
        .filter(col("size") == "S") \
        .groupBy("category", "size", "pizza_id") \
        .agg(round(sum(col("price") * col("quantity")), 2).alias("Total_Sales")) \
        .selectExpr("pizza_id AS Pizza_Code", "category", "Total_Sales AS Total_Amount") \
        .orderBy("category", desc("Total_Amount"), "Pizza_Code")
  

        ScenarioFiveDf.display()
        return ScenarioFiveDf
    

    def requirementSix(self,inputs):

        df = Derivate.derivate(self,inputs)


        ScenarioSixDf = df\
            .filter((col("Order_Month") == 5) & (col("category") == "Chicken"))\
            .groupBy("category","Order_Month","name")\
            .agg(round(sum(col("price")* col("quantity")),2).alias("Total_Sales"))\
            .selectExpr("name AS Pizza_Name","Total_Sales")\
            .orderBy("Pizza_Name") 


       # ScenarioSixDf = spark.sql("select name,round(sum(("price")*("quantity")),2)")
        
        ScenarioSixDf.display()
        return ScenarioSixDf

    def requirementSeven(self,inputs):

        df = Derivate.derivate(self,inputs)

        ScenarioSevenDf =  df \
            .groupBy("name","Order_Month")\
            .agg(round(sum(col("price") * col("quantity")),2).alias("Total_Sales")) \
            .selectExpr("name as Pizza_Name","Order_Month","Total_Sales")\
            .orderBy("Order_Month",desc("Total_Sales"),"Pizza_Name")

        ScenarioSevenDf.display()
        return ScenarioSevenDf